In [1]:
import nltk
import json
import random

## Carga de los datos

Punto de partida: un .json con una lista de proyectos asignados 

In [2]:
with open('../tmp/devex-corpus.json') as corpus_file:
    corpus = json.loads(corpus_file.read())

In [3]:
print(type(corpus))
print(corpus[0])

<class 'list'>
["GE-Tbilisi: Legislative impact assessment, drafting and representationService contract award noticeGeorgia1.Publication reference:EuropeAid/135035/DH/SER/GE.2.Publication date of the contract notice:30.4.2014.3.Lot number and lot title:1 lot only.4.Contract number and value:ENPI/2015/357121, 1 713 780 EUR.5.Date of award of the contract:23.1.2015.6.Number of tenders received:6.7.Overall score of chosen tender:99,54.8.Name, address and nationality of successful tenderer:IBF International Consulting SA, avenue Louise 209A, 1050 Brussels, BELGIUM.Center for International Legal Cooperation (CILC), Koninginnegracht 7, 2514 AA The Hague, NETHERLANDS.Deutsche Stiftung fűr internationale rechtliche Zusammenarbeit e.V. (IRZ), Ubierstraße 92, 53173 Bonn, GERMANY.ISO code of country: BE.9.Duration of contract:30 months.10.Contracting authority:European Union, represented by the European Commission, on behalf of and for the account of the partner country, Tbilisi, GEORGIA.11.Legal

## Preparado de los datos

En primer lugar, vamos a convertir las descripciones en listas de palabras.

In [4]:
descriptions = []
for description, company in corpus:
    words = description.split()
    descriptions.append((words, company))

A partir de las descripciones, tenemos que extraer una lista de palabras características ("word features"). Será simplemente la lista total de palabras que aparecen en nuestras descripciones. Usamos las siguientes funciones:

In [5]:
def get_words_in_descriptions(descriptions):
    """
    Given a list of descriptions [([description words], "company name")], join all the
    words into a single list and return it.
    """
    all_words = []
    for (words, company) in descriptions:
      all_words.extend(words)
    return all_words

In [6]:
def get_word_features(wordlist):
    """Given a list of all the words, return a (dict_keys) list of unique words."""
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

(Una **FreqDist** es simplemente una cuenta de cuántas veces aparece cada palabra)

In [7]:
wordlist = nltk.FreqDist(get_words_in_descriptions(descriptions))
wordlist.most_common(10)

[('of', 4575),
 ('the', 4431),
 ('and', 4298),
 ('to', 2443),
 ('-', 2184),
 ('Modification', 1808),
 ('DURATION:', 1692),
 ('in', 1476),
 ('ACTION', 1438),
 ('LOCATION:', 1437)]

In [8]:
word_features = get_word_features(get_words_in_descriptions(descriptions))

## Creación del clasificador

Nuestro clasificador será una caja negra. Le proporcionaremos una descripción, y nos devolverá una clasificación (la empresa que más probable ganaría este contrato).

Nuestro clasificador necesitará un training set. Usaremos las primeras 500 descripciones como training set, y el resto como testing set.

In [9]:
# es importante tener grupos de training y testing homogéneos (mismas proporciones de cada empresa)
random.seed(54965)
random.shuffle(descriptions)

In [10]:
training_set = descriptions[:500]
testing_set = descriptions[500:]

Para el clasificador no podemos usar nuestras descripciones directamente. Necesitamos una función (*feature extractor*) que sea capaz de extraer las features de la descripción. Esta función solo toma la descripción y devuelve nuestras *word features* indicando si son relevantes (**aproximación más básica: si aparecen o no en la descripción**).

In [11]:
def extract_features(description):
    """
    Feature extractor. Given a description (as a list of words), returns our list of word features,
    indicating whether each word is relevant for the description.
    """
    # remove duplicates
    description_words = set(description)
    features = {}
    # word_features is defined outside
    for word in word_features:
        features['contains({})'.format(word)] = (word in description_words)
    return features

In [12]:
extract_features('this is a project made in GUATEMALA'.split())

{'contains(reach—headquartered)': False,
 'contains(Tyumen,)': False,
 'contains(Tracts,)': False,
 'contains(EQUATE)': False,
 'contains(123)': False,
 'contains(85,14.)': False,
 'contains(Gottschalk.)': False,
 'contains(HVB)': False,
 'contains(FROM:)': False,
 'contains(850)': False,
 'contains(Bishkek)': False,
 'contains(Scoping)': False,
 'contains((Khon)': False,
 'contains((IVL))': False,
 'contains(Activities)': False,
 'contains(supplier.)': False,
 'contains(84)': False,
 'contains(Direction)': False,
 'contains(prone)': False,
 'contains(TRAGSA)': False,
 'contains((initial)': False,
 'contains(2011-05-18:)': False,
 'contains(SUR)': False,
 'contains(removes)': False,
 'contains(program.)': False,
 'contains(dietary)': False,
 'contains(students)': False,
 'contains(Hssen,)': False,
 'contains(proven)': False,
 'contains(géographique:)': False,
 'contains(15161.)': False,
 'contains(650000)': False,
 'contains(Ndjamena)': False,
 'contains(Antipolis;)': False,
 'contains

Usamos esta función para crear el training set, convirtiendo las descripciones en listas de features entendibles por el clasificador:

In [13]:
training_set = nltk.classify.apply_features(extract_features, training_set)
testing_set = nltk.classify.apply_features(extract_features, testing_set)

finalmente, creamos el clasificador, usando nuestro training set para entrenarlo

In [14]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [15]:
classifier.show_most_informative_features()

Most Informative Features
            contains(EC) = True             UNDP : IBF In =     29.1 : 1.0
     contains(DURATION:) = False          Chemon : AGRECO =     29.0 : 1.0
contains(CONTRACTACTIONTYPE:) = True           Chemon : Ernst  =     29.0 : 1.0
  contains(Modification) = True           Chemon : Ernst  =     29.0 : 1.0
         contains(FROM:) = True           Chemon : Ernst  =     29.0 : 1.0
       contains(FUNDING) = True           Chemon : Ernst  =     29.0 : 1.0
contains(PRODUCTORSERVICECODE:) = True           Chemon : Ernst  =     29.0 : 1.0
      contains(PSC_CAT:) = True           Chemon : Ernst  =     29.0 : 1.0
contains(DESCRIPTIONOFCONTRACTREQUIREMENT:) = True           Chemon : Ernst  =     29.0 : 1.0
      contains(services) = True           Chemon : Ernst  =     28.2 : 1.0


## Application of the classifier

Ahora podemos probar el clasificador. Intentemos clasificar una sola descripción:

In [16]:
"El primer contrato ha sido asignado a la empresa {}".format(testing_set[0][1])

'El primer contrato ha sido asignado a la empresa Chemonics International Inc.'

Comprobemos si nuestro clasificador lo hace bien:

In [17]:
classifier.classify(testing_set[0][0])

'Ernst & Young'

¡Uhm! Pero, ¿qué hay del resto? Creamos una función para comprobar los aciertos en **todo** el training set:

In [18]:
def check_accuracy(training_set, classifier):
    """Given a training set and a classifier, apply it to all cases and return the percentage of hits."""
    hits = 0
    cases = 0
    for description, company in training_set:
        if classifier.classify(description) == company:
            hits += 1
        cases += 1
    return hits/cases

In [19]:
accuracy = check_accuracy(training_set, classifier)
print("Nuestro clasificador acierta en el {}% de los casos".format(accuracy*100))

Nuestro clasificador acierta en el 39.800000000000004% de los casos
